In [2]:
import tsl
import numpy as np

In [3]:
import pandas as pd
import geopandas as gpd

path_file = '../netmob2023/pytorch/data/raw_ts/Paris Instagram cell hourly aggregated.parquet'
data = pd.read_parquet(path_file)
data_paris_inst = data.loc['Paris','Instagram']
display(data_paris_inst.head)

<bound method NDFrame.head of          2019-03-16 00:00:00  2019-03-16 01:00:00  2019-03-16 02:00:00  \
tile_id                                                                  
162                   184968               151543                45170   
507                   206487               193399                55599   
508                   198946               172370                50789   
853                   200217               189045                54992   
854                   197813               171446                51680   
...                      ...                  ...                  ...   
141127                 24672                18345                 4916   
141128                 28131                19775                 4677   
141129                 30136                20422                 4415   
141130                 31322                21264                 4405   
141476                 32909                21937                 4465   

       

In [4]:
list_cities = ['Paris']
mapping = []
for city in list_cities :
    grid = gpd.read_parquet(f'../netmob2023/pytorch/data/grid cells/NETMOB grids/{city} grid.parquet').set_index(drop = True, keys = 'tile_id')
    IRIS = gpd.read_parquet(f'../netmob2023/pytorch/data/grid cells/IRIS grids/IRIS {city}.parquet').drop(columns = ["code_commune"]).set_index(drop = True, keys = 'code_iris')
    #display(grid)
    #display(IRIS)

    # Compute the spatial join between NETMOB cells and IRIS cells, according to the 'intersect' predicate.
    join = grid.sjoin(IRIS, how = 'left', predicate = 'intersects')

    # Remove the NETMOB cells that do not intersect any IRIS cell.
    num_cells_noIRIS = len(join.loc[join['index_right'].isna()])
    num_cells = len(join.index.get_level_values('tile_id').unique())
    print(f"{city}, # NETMOB cells without any IRIS association: {num_cells_noIRIS}/{num_cells}")
    join = join.loc[~join['index_right'].isna()]


    # Prepare the final version of the dataframe. 
    join.reset_index(inplace = True)
    join.drop('geometry', axis = 1, inplace = True) # Drop the geometry column.
    join.rename(columns = {'index_right':'IRIS_id'}, inplace = True)
    join.set_index(drop = True, keys = ['IRIS_id'], inplace = True)
    join = pd.concat([join], keys=[city], names=['city']) # Add a new level on top of the existing multi-index, indicating the name of the city.

    # display(join)
    mapping.append(join)

final = pd.concat(mapping)
display(final)
final.to_parquet(f'mapping IRIS-NETMOB cells.parquet')

Paris, # NETMOB cells without any IRIS association: 223/81731


tile_id
city  IRIS_id           
Paris 916870110      162
      916870110      507
      916870110      508
      916870112      853
      916870110      853
...                  ...
      930730101   141127
      930730101   141128
      930730101   141129
      930730101   141130
      930730101   141476

[122882 rows x 1 columns]

In [5]:
import geopandas as gpd
import pandas as pd

list_cities = ['Paris']
mapping = []

for city in list_cities:
    grid = gpd.read_parquet(f'../netmob2023/pytorch/data/grid cells/NETMOB grids/{city} grid.parquet').set_index(drop=True, keys='tile_id')
    IRIS = gpd.read_parquet(f'../netmob2023/pytorch/data/grid cells/IRIS grids/IRIS {city}.parquet').drop(columns=["code_commune"]).set_index(drop=True, keys='code_iris')

    # Compute the spatial join between NETMOB cells and IRIS cells, according to the 'intersect' predicate.
    join = grid.sjoin(IRIS, how='left', predicate='intersects')

    # Remove the NETMOB cells that do not intersect any IRIS cell.
    num_cells_noIRIS = len(join.loc[join['index_right'].isna()])
    num_cells = len(join.index.get_level_values('tile_id').unique())
    print(f"{city}, # NETMOB cells without any IRIS association: {num_cells_noIRIS}/{num_cells}")
    join = join.loc[~join['index_right'].isna()]

    # Calculate the area of intersection for each cell.
    join['area'] = join.apply(lambda row: row['geometry'].intersection(IRIS.loc[row['index_right']].geometry).area, axis=1)

    # Prepare the final version of the dataframe. 
    join.reset_index(inplace=True)
    join.drop('geometry', axis=1, inplace=True)  # Drop the geometry column.
    join.rename(columns={'index_right': 'IRIS_id'}, inplace=True)
    join.set_index(drop=True, keys=['IRIS_id'], inplace=True)
    join = pd.concat([join], keys=[city], names=['city'])  # Add a new level on top of the existing multi-index, indicating the name of the city.

    mapping.append(join)

final = pd.concat(mapping)
display(final.loc['Paris'])
final = final.loc['Paris']
final.to_parquet(f'mapping IRIS-NETMOB cells.parquet')


Paris, # NETMOB cells without any IRIS association: 223/81731


,tile_id,area
IRIS_id,,
916870110,162,6.084022e-08
916870110,507,5.262143e-07
916870110,508,1.133561e-06
916870112,853,6.803781e-07
916870110,853,5.390065e-07
...,...,...
930730101,141127,1.227604e-06
930730101,141128,1.227604e-06
930730101,141129,1.227604e-06


In [6]:
import geopandas as gpd
import pandas as pd

city = 'Paris'
mapping = []

grid = gpd.read_parquet(f'../netmob2023/pytorch/data/grid cells/NETMOB grids/{city} grid.parquet').set_index('tile_id')
IRIS = gpd.read_parquet(f'../netmob2023/pytorch/data/grid cells/IRIS grids/IRIS {city}.parquet').drop(columns=["code_commune"]).set_index('code_iris')

# Ensure both datasets are in the same projected CRS
projected_crs = "EPSG:32631"  # UTM Zone 31N, suitable for Paris

if grid.crs != projected_crs:
    grid = grid.to_crs(projected_crs)
if IRIS.crs != projected_crs:
    IRIS = IRIS.to_crs(projected_crs)


# Compute the spatial join between NETMOB cells and IRIS cells, according to the 'intersect' predicate.
join = grid.sjoin(IRIS, how='left', predicate='intersects')

# Remove the NETMOB cells that do not intersect any IRIS cell.
num_cells_noIRIS = len(join[join['index_right'].isna()])
num_cells = grid.shape[0]
print(f"{city}, # NETMOB cells without any IRIS association: {num_cells_noIRIS}/{num_cells}")
join = join[~join['index_right'].isna()]

# Calculate the area of intersection for each cell.
join['area'] = join.apply(lambda row: row['geometry'].intersection(IRIS.loc[row['index_right']].geometry).area/10000, axis=1)

# Prepare the final version of the dataframe.
join.reset_index(inplace=True)
join.drop('geometry', axis=1, inplace=True)  # Drop the geometry column.
join.rename(columns={'index_right': 'IRIS_id'}, inplace=True)
join.set_index('IRIS_id', inplace=True)
join = pd.concat([join], keys=[city], names=['city'])  # Add a new level on top of the existing multi-index, indicating the name of the city.

mapping.append(join)

final = pd.concat(mapping)
display(final.loc[city])

# Verify that the sum of intersection areas equals the area of each NETMOB cell.
final = final.loc[city]

# Calculate the area of each NETMOB cell.
grid['area'] = grid['geometry'].area/10000

# Merge the NETMOB grid area information with the final mapping.
merged = final.reset_index().merge(grid[['area']], left_on='tile_id', right_index=True, how='left', suffixes=('', '_netmob'))

# Group by the NETMOB cell ID and sum the intersection areas.
intersection_sums = merged.groupby('tile_id')['area'].sum()

# Compare the summed intersection areas with the original NETMOB cell areas.
comparison = pd.DataFrame({'netmob_area': merged.drop_duplicates('tile_id').set_index('tile_id')['area_netmob'],
                           'intersection_sum': intersection_sums})

# Check if they are equal (with a tolerance for floating point errors).
comparison['matches'] = (comparison['netmob_area'] - comparison['intersection_sum']).abs() < 0.1

# Output the comparison for inspection.
display(comparison)

# Save the final mapping to a parquet file.
final.to_parquet('mapping IRIS-NETMOB cells.parquet')

# Count the number of matches.
num_matches = comparison['matches'].sum()
print(f"Number of matches: {num_matches}/{comparison.shape[0]}")


Paris, # NETMOB cells without any IRIS association: 223/81731


,tile_id,area
IRIS_id,,
916870110,162,0.049818
916870110,507,0.430873
916870110,508,0.928181
916870112,853,0.557096
916870110,853,0.441342
...,...,...
930730101,141127,0.997952
930730101,141128,0.997952
930730101,141129,0.997952


,netmob_area,intersection_sum,matches
tile_id,,,
162,0.998439,0.049818,False
507,0.998438,0.430873,False
508,0.998438,0.928181,True
853,0.998437,0.998437,True
854,0.998437,0.998437,True
...,...,...,...
141127,0.997952,0.997952,True
141128,0.997952,0.997952,True
141129,0.997952,0.997952,True


Number of matches: 80562/81508


In [7]:
final

,tile_id,area
IRIS_id,,
916870110,162,0.049818
916870110,507,0.430873
916870110,508,0.928181
916870112,853,0.557096
916870110,853,0.441342
...,...,...
930730101,141127,0.997952
930730101,141128,0.997952
930730101,141129,0.997952


In [8]:
data_paris_inst

,2019-03-16 00:00:00,2019-03-16 01:00:00,2019-03-16 02:00:00,2019-03-16 03:00:00,2019-03-16 04:00:00,2019-03-16 05:00:00,2019-03-16 06:00:00,2019-03-16 07:00:00,2019-03-16 08:00:00,2019-03-16 09:00:00,...,2019-05-31 14:00:00,2019-05-31 15:00:00,2019-05-31 16:00:00,2019-05-31 17:00:00,2019-05-31 18:00:00,2019-05-31 19:00:00,2019-05-31 20:00:00,2019-05-31 21:00:00,2019-05-31 22:00:00,2019-05-31 23:00:00
tile_id,,,,,,,,,,,,,,,,,,,,,
162,184968,151543,45170,22154,29494,39268,30488,55808,125310,149488,...,239079,180847,157401,218383,305176,237662,203873,229719,193062,190817
507,206487,193399,55599,23874,30676,54293,36662,53060,152353,153309,...,262779,187160,187571,250310,335123,214496,215639,274825,210080,274394
508,198946,172370,50789,25020,32070,47528,33207,53952,140262,151890,...,252086,182860,174550,237193,317257,227113,210877,252390,201230,238024
853,200217,189045,54992,23829,30838,54184,35393,49371,148172,143221,...,248414,176140,181520,240200,317458,195321,204771,265695,198353,273316
854,197813,171446,51680,26550,33447,47917,32310,50624,139675,147339,...,242725,174294,173179,232473,301050,211719,203692,249780,193916,245852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141127,24672,18345,4916,10438,10800,80025,101256,68958,73965,83384,...,104241,86411,111828,111151,96992,87984,135842,108747,96356,72597
141128,28131,19775,4677,10845,12365,92447,117521,79000,83904,94492,...,121448,99534,126878,126513,111627,101625,153996,125052,110141,83231
141129,30136,20422,4415,10940,12902,100022,124707,81730,86228,97211,...,126428,103108,130488,130565,115522,105137,159801,129789,114197,87779


In [9]:
final['tile_id'] = final['tile_id'].astype(int)
data_paris_inst.index = data_paris_inst.index.astype(int)


# Merge the 'final' DataFrame with 'data' to include the non-IRIS cell data
merged = final.reset_index().merge(data_paris_inst, left_on='tile_id', right_index=True, how='left')

# Scale the 'data' rows by the intersection area
for column in data.columns:
    merged[column] = merged[column] * merged['area']

# Sum the scaled rows for each IRIS cell
IRIS_data = merged.groupby('IRIS_id')[data.columns].sum()

# Display the resulting IRIS_data DataFrame
display(IRIS_data)

# Save the IRIS_data DataFrame to a parquet file
IRIS_data.to_parquet('IRIS_data.parquet')

,2019-03-16 00:00:00,2019-03-16 01:00:00,2019-03-16 02:00:00,2019-03-16 03:00:00,2019-03-16 04:00:00,2019-03-16 05:00:00,2019-03-16 06:00:00,2019-03-16 07:00:00,2019-03-16 08:00:00,2019-03-16 09:00:00,...,2019-05-31 14:00:00,2019-05-31 15:00:00,2019-05-31 16:00:00,2019-05-31 17:00:00,2019-05-31 18:00:00,2019-05-31 19:00:00,2019-05-31 20:00:00,2019-05-31 21:00:00,2019-05-31 22:00:00,2019-05-31 23:00:00
IRIS_id,,,,,,,,,,,,,,,,,,,,,
751010101,3.991105e+07,2.769814e+07,1.616417e+07,1.126950e+07,7.394045e+06,5.760579e+06,3.859169e+06,1.250086e+07,1.675009e+07,2.582332e+07,...,6.361809e+07,6.556237e+07,6.380223e+07,6.458502e+07,7.106878e+07,6.179063e+07,4.956028e+07,4.592108e+07,5.320973e+07,3.757532e+07
751010102,3.068762e+07,1.944489e+07,1.238866e+07,9.070445e+06,6.851605e+06,4.514784e+06,3.619569e+06,1.018061e+07,1.289382e+07,1.807200e+07,...,4.537162e+07,4.562175e+07,4.846506e+07,4.597854e+07,5.030873e+07,4.373148e+07,3.966678e+07,3.952204e+07,4.066110e+07,3.324053e+07
751010103,1.817594e+07,1.254729e+07,8.510844e+06,5.277046e+06,3.592954e+06,3.282004e+06,2.376932e+06,5.841095e+06,7.749232e+06,1.230358e+07,...,2.888610e+07,3.018739e+07,3.010605e+07,2.920094e+07,3.051814e+07,2.692204e+07,2.203360e+07,2.273048e+07,2.309507e+07,1.799745e+07
751010104,7.448092e+07,5.061377e+07,3.823598e+07,2.646194e+07,1.842436e+07,9.739640e+06,1.208289e+07,1.956247e+07,3.375916e+07,4.730346e+07,...,1.199803e+08,1.227520e+08,1.239150e+08,1.334654e+08,1.247981e+08,1.126252e+08,8.748956e+07,8.114465e+07,7.934425e+07,6.530749e+07
751010105,7.307844e+07,4.345666e+07,3.440809e+07,2.737979e+07,1.971387e+07,9.835191e+06,8.550031e+06,1.851694e+07,3.615188e+07,3.897027e+07,...,1.637236e+08,1.584700e+08,1.770841e+08,1.711805e+08,1.478306e+08,1.243561e+08,8.855233e+07,6.729356e+07,8.758419e+07,6.296891e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950180902,2.214216e+07,1.612729e+07,1.510914e+07,8.355471e+06,3.232700e+06,2.796289e+06,3.744607e+06,1.260162e+07,1.367835e+07,1.884612e+07,...,2.012168e+07,1.676465e+07,2.298804e+07,2.218224e+07,2.175166e+07,2.806351e+07,2.371037e+07,2.321253e+07,2.322683e+07,1.829096e+07
950180903,1.815223e+07,1.334159e+07,8.414547e+06,5.515380e+06,4.633554e+06,2.251098e+06,3.171123e+06,7.813514e+06,1.267316e+07,1.696514e+07,...,1.864190e+07,1.856673e+07,2.076081e+07,2.142562e+07,2.325441e+07,2.471430e+07,2.529699e+07,2.461384e+07,2.119041e+07,1.910346e+07
950181001,2.570030e+07,1.787190e+07,1.034853e+07,7.488501e+06,7.114080e+06,4.806210e+06,5.011036e+06,7.722206e+06,1.636402e+07,1.780742e+07,...,1.777044e+07,2.497711e+07,2.063044e+07,1.982176e+07,2.358916e+07,2.915415e+07,2.657897e+07,2.866696e+07,2.690740e+07,2.530774e+07


In [10]:
IRIS_data = IRIS_data.sample(n=200, random_state=42) 

In [11]:
IRIS_data = IRIS_data.transpose()
IRIS_data.index.name = None


In [12]:
IRIS_data


IRIS_id,751135014,930730301,920120301,920070108,920230202,751124611,751207917,930290602,751155703,930590107,...,751186904,751135103,920240102,940810105,751135124,920360203,751155807,751187107,751145619,751155815
2019-03-16 00:00:00,2.273138e+07,1.242545e+07,1.029895e+07,6.094632e+06,1.504298e+07,1.396567e+07,1.803923e+07,1.551548e+07,2.617957e+07,2.613417e+07,...,9.886927e+07,9.496510e+06,1.218933e+07,8.505738e+06,2.028343e+07,8.333896e+06,5.366553e+07,1.253361e+07,2.173774e+07,1.424467e+07
2019-03-16 01:00:00,1.491366e+07,1.429730e+07,7.344583e+06,3.091782e+06,1.099698e+07,1.107212e+07,1.110984e+07,7.278345e+06,1.499801e+07,1.286286e+07,...,7.025247e+07,5.870225e+06,5.911744e+06,5.519331e+06,1.293259e+07,5.687149e+06,3.438551e+07,9.442131e+06,1.253463e+07,1.023088e+07
2019-03-16 02:00:00,8.460244e+06,6.126175e+06,8.098362e+06,2.166591e+06,4.415509e+06,6.966169e+06,7.955861e+06,4.730484e+06,1.160325e+07,1.233480e+07,...,4.153700e+07,3.542913e+06,4.139310e+06,2.577938e+06,8.554362e+06,3.318808e+06,2.195700e+07,5.668945e+06,7.134156e+06,7.977582e+06
2019-03-16 03:00:00,5.718642e+06,1.869241e+06,4.551185e+06,1.329462e+06,4.323397e+06,4.663288e+06,3.937131e+06,1.881540e+06,6.124520e+06,8.638769e+06,...,2.124141e+07,2.980391e+06,3.003509e+06,1.588357e+06,5.916970e+06,5.200534e+06,1.339537e+07,5.065905e+06,2.570298e+06,5.170278e+06
2019-03-16 04:00:00,2.795014e+06,2.300844e+06,3.217181e+06,1.669605e+06,3.793054e+06,4.096109e+06,2.425453e+06,1.709721e+06,5.770561e+06,8.604554e+06,...,2.115543e+07,2.028552e+06,2.660271e+06,8.043276e+05,4.111186e+06,2.939318e+06,1.067799e+07,2.113254e+06,3.972926e+06,5.240701e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-05-31 19:00:00,2.133097e+07,1.656582e+07,1.524611e+07,9.439199e+06,1.832209e+07,1.470577e+07,1.267177e+07,8.071424e+06,2.126199e+07,2.048442e+07,...,9.969047e+07,7.526339e+06,1.350197e+07,8.164960e+06,2.110909e+07,9.697547e+06,5.471200e+07,1.279264e+07,1.445465e+07,1.403687e+07
2019-05-31 20:00:00,2.376270e+07,2.117691e+07,1.305310e+07,8.996184e+06,1.669758e+07,1.125275e+07,1.344729e+07,1.085967e+07,2.270593e+07,1.693133e+07,...,9.404710e+07,7.723422e+06,1.296237e+07,1.153483e+07,2.338498e+07,8.361741e+06,5.126057e+07,1.156856e+07,1.365393e+07,1.269412e+07
2019-05-31 21:00:00,2.254518e+07,1.831641e+07,1.177799e+07,7.641625e+06,1.546652e+07,1.022662e+07,1.289215e+07,1.054563e+07,2.434036e+07,1.530013e+07,...,9.330272e+07,7.739006e+06,1.339001e+07,1.123384e+07,1.971295e+07,1.006200e+07,5.075900e+07,1.518074e+07,2.190177e+07,1.292267e+07
2019-05-31 22:00:00,1.831064e+07,1.801348e+07,1.342091e+07,7.550257e+06,1.792307e+07,1.232194e+07,1.044576e+07,1.676574e+07,1.987701e+07,2.054705e+07,...,7.166533e+07,9.956925e+06,1.303135e+07,1.117262e+07,1.965065e+07,7.683102e+06,5.167469e+07,1.626284e+07,2.175426e+07,9.383293e+06


In [13]:
from tsl.data import SpatioTemporalDataset

torch_dataset = SpatioTemporalDataset(target=IRIS_data,
                                      horizon = 12)



torch_dataset.set_connectivity(np.ones((torch_dataset.n_nodes, torch_dataset.n_nodes),dtype=np.int64),target_layout='edge_index')

In [14]:
torch_dataset

SpatioTemporalDataset(n_samples=1825, n_nodes=200, n_channels=1)

In [15]:
torch_dataset.n_nodes

200

In [16]:
from tsl.nn.models import GraphWaveNetModel

gwn = GraphWaveNetModel(input_size=1,
                        output_size=1,
                        horizon=12,
                        n_nodes = torch_dataset.n_nodes,
                        learned_adjacency=True)

In [17]:
from tsl.data.datamodule import (SpatioTemporalDataModule,
                                 TemporalSplitter)
from tsl.data.preprocessing import StandardScaler

# Normalize data using mean and std computed over time and node dimensions
scalers = {'target': StandardScaler(axis=(0, 1))}

# Split data sequentially:
#   |------------ dataset -----------|
#   |--- train ---|- val -|-- test --|
splitter = TemporalSplitter(val_len=0.1, test_len=0.2)

dm = SpatioTemporalDataModule(
    dataset=torch_dataset,
    scalers=scalers,
    splitter=splitter,
    batch_size=64,
)

print(dm)

SpatioTemporalDataModule(train_len=None, val_len=None, test_len=None, scalers=[target], batch_size=64)


In [18]:
dm.setup()
print(dm)

SpatioTemporalDataModule(train_len=1302, val_len=134, test_len=365, scalers=[target], batch_size=64)


In [19]:
from tsl.metrics.torch import MaskedMAE, MaskedMAPE
from tsl.engines import Predictor
from torch.optim import Adam

loss_fn = MaskedMAE()

metrics = {'mae': MaskedMAE(),
           'mape': MaskedMAPE(),
           'mae_at_15': MaskedMAE(at=2),  # '2' indicates the third time step,
                                          # which correspond to 15 minutes ahead
           'mae_at_30': MaskedMAE(at=5),
           'mae_at_60': MaskedMAE(at=11)}

# setup predictor
predictor = Predictor(
    model=gwn,                   # our initialized model
    optim_class=Adam,  # specify optimizer to be used...
    optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
    loss_fn=loss_fn,               # which loss function to be used
    metrics=metrics                # metrics to be logged during train/val/test
)

In [20]:
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger(save_dir="logs", name="tsl_intro", version=0)

In [22]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    dirpath='logs',
    save_top_k=1,
    monitor='val_mae',
    mode='min',
)

trainer = pl.Trainer(max_epochs=10,
                     logger=logger,
                     accelerator="cpu",
                     limit_train_batches=100,  # end an epoch after 100 updates
                     callbacks=[checkpoint_callback])

trainer.fit(predictor, datamodule=dm)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/gloddi/anaconda3/envs/tsl/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/gloddi/anaconda3/envs/tsl/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:652: Checkpoint directory /Users/gloddi/Documents/GitHub/tsl/logs exists and is not empty.

  | Name          | Type              | Params | Mode 
------------------------------------------------------------
0 | loss_fn       | MaskedMAE         | 0      | train
1 | train_metrics | MetricCollection  | 0      | train
2 | val_metrics   | MetricCollection  | 0      | train
3 | test_metrics  | MetricCollection  | 0      | train
4 | model         | GraphWaveNetModel | 301 K  | train
------------------------------------------------------------
301 K     Trainable params
0         Non-train

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/gloddi/anaconda3/envs/tsl/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
Only args ['edge_weight', 'x', 'edge_index'] are forwarded to the model (GraphWaveNetModel).
/Users/gloddi/anaconda3/envs/tsl/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/gloddi/anaconda3/envs/tsl/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (20) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to s

Training: |          | 0/? [00:00<?, ?it/s]

/Users/gloddi/anaconda3/envs/tsl/lib/python3.12/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
